In [21]:
import pandas as pd

In [22]:
df1 = pd.read_csv('../data/df.csv')
df2 = pd.read_csv('../data/festivalcharac.csv', encoding = 'euc-kr')

In [23]:
# 공백삭제
df1['name_year'] = df1['name_year'].str.replace(' ', '', regex=True)

In [24]:
# '그룹명'과 '구분명' col의 category 데이터들을 깔끔하게 정리하기.
df2['그룹명'] = df2['그룹명'].replace({
    '축제기간': 'Fe',
    '비축제기간': 'non'
})

df2['구분명'] = df2['구분명'].replace({
    '외부방문자 유입': 'foreigner',
    '현지인방문자 유입': 'local',
    '내비게이션 검색량': 'navi',
    '관광소비': 'tour_fee',
    '축제지 집중률': 'festival_conc'
})

# 결측치 처리. 상대적인 비율이라 0으로 표기된 부분이 있을 순 있음.
df2['구분값'] = df2['구분값'].replace(0, 0.000001)
# 새로운 column 이름이 될 예비 열 생성
df2['sum'] = df2['그룹명'] + '_' + df2['구분명']

# 피봇 테이블
pivot_df2 = df2.pivot_table(index=df2['index'], columns='sum', values='구분값', aggfunc=lambda x: x)

# 모두가 0이었던 결측치 드랍.
df2 = pivot_df2.loc[~(pivot_df2 == 0.000001).all(axis=1)].reset_index()
df2['year'] = df2['index'].str[-4:].astype(int)


In [25]:
#rapidfuzz로 이름이 약간 다른 파일명이라도 매칭해서 합치기.

from rapidfuzz import process as rf_process
from rapidfuzz import fuzz

def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower().replace(" ", "")

df1['normalized_index'] = df1['name_year'].apply(normalize_text)
df2['normalized_index'] = df2['index'].apply(normalize_text)

matches = []
for _, row1 in df1.iterrows():
    match = rf_process.extractOne(
        row1['normalized_index'],
        df2['normalized_index'],
        scorer=fuzz.ratio
    )
    if match and match[1] >= 75:  
        matched_row = df2[df2['normalized_index'] == match[0]]
        if not matched_row.empty:
            matched_row = matched_row.iloc[0]  
            if (
                row1['year'] == matched_row['year']):
                    matches.append({
                    'df1_index': row1['name_year'],
                    'df2_index': matched_row['index'],
                    'similarity_score': match[1],
                    'year': row1['year']
                })

matched_df = pd.DataFrame(matches)


In [26]:

df2_matched = pd.merge(
    df1, 
    matched_df[['df1_index', 'df2_index']], 
    left_on='name_year',  
    right_on='df1_index',  
    how='left'  
)


final_df = pd.merge(
    df2_matched,  
    df2,  
    left_on='df2_index',  
    right_on='index', 
    how='left' 
)

print(final_df.head())              #fuzz 한 후 드랍할 column들 정리.
final_df = final_df.drop(columns = ['normalized_index_y',
                                    'normalized_index_x',
                                    'year_y',
                                    'df1_index',
                                    'df2_index', 
                                    'index',
                                    'Unnamed: 0'])

final_df.rename(columns = {'year_x' : 'year'}, inplace = True)

final_df.to_csv('../data/df_charac.csv')


   Unnamed: 0 Festival   name_year  month  year_x  visitors          cost  \
0           0   강경젓갈축제  강경젓갈축제2018     10    2018    110041  8.250000e+08   
1           1   강경젓갈축제  강경젓갈축제2022     10    2022    127900  1.003000e+09   
2           2   강경젓갈축제  강경젓갈축제2023     10    2023    124299  1.178086e+09   
3           3   강릉커피축제  강릉커피축제2018     10    2018    144268  3.904450e+08   
4           4   강릉커피축제  강릉커피축제2019     10    2019    160896  4.400000e+08   

         date  visit/cost  target  ...  Fe_local   Fe_navi Fe_tour_fee  \
0  2018-10-01    0.000133     old  ...  0.921656  0.571892    0.616762   
1  2022-10-01    0.000128     old  ...  0.875192  0.635511    0.722520   
2  2023-10-01    0.000106     old  ...  0.951465  0.662677    0.684069   
3  2018-10-01    0.000369  family  ...  0.659793  0.446371    0.582128   
4  2019-10-01    0.000366  family  ...  0.860715  0.495819    0.579160   

  non_festival_conc  non_foreigner  non_local  non_navi  non_tour_fee  year_y  \
0          

In [27]:
df3.columns

Index(['축제명', '개최년도', '축제기간(일)', '(현지인)방문자수', '(외지인)방문자수', '(외국인)방문자수',
       '(전체)방문자수', '일평균 방문자수', '전년도 일평균 방문자수', '일평균 방문자수 증감률', '(이전)전체방문자',
       '(전체)방문자증감', '(현지인)방문자비율', '(외지인)방문자비율', '(외국인)방문자비율', '전년대비방문자증감비율',
       'index_x', '도', '시군구', 'normalized_index_x', 'df1_index', 'df2_index',
       '회계연도', '지역명', '행사축제명', '총비용', '사업수익', '순원가', '시작월', '종료월', 'index_y',
       'normalized_index_y'],
      dtype='object')

In [28]:
df3 = pd.read_csv('../data/전국연도별방문자회계정보정말정말최종ver.csv')

final_df = pd.merge(
    final_df,  
    df3[['축제기간(일)', '도', 'index_y']],  
    left_on='name_year',  
    right_on='index_y', 
    how='left' 
)


In [29]:

final_df.to_csv('../data/df_charac.csv')